In [1]:
# Dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Read CSV, place in DF
df = pd.read_csv("features.csv")
df.head()
df = df.dropna()

In [3]:
heartWillGoOn = [{'acousticness': 0.732, 'danceability': 0.428, 'energy': 0.276,  
                 'id': '3oEHQmhvFLiE7ZYES0ulzv', 'instrumentalness': 5.33e-06, 'key': 4, 
                 'liveness': 0.117,'loudness': -11.729,'mode': 1,
                  'speechiness': 0.0312, 'tempo': 99.195, 'valence': 0.0382}]

In [4]:
df = df.append(heartWillGoOn)
df = df.reset_index()
df.tail()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,index,Unnamed: 0,acousticness,danceability,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
143674,143675,143675.0,0.000157,0.634,0.958,4WYZ2dp1BhnPovdiXsvU2S,0.702000,9,0.0898,-5.501,0,0.0340,140.015,0.8140
143675,143676,143676.0,0.082000,0.507,0.849,1biP5S4CFMNfLA0rceCmRF,0.000000,5,0.7970,-9.197,1,0.1040,132.953,0.1540
143676,143677,143677.0,0.000821,0.681,0.942,3DZpiHNi3wQuoIMuiJ5jxJ,0.000030,9,0.0656,-4.475,1,0.0483,131.965,0.8850
143677,143678,143678.0,0.033900,0.834,0.869,3MYe1siYBRujEK5fy9aa3h,0.000522,9,0.1460,-5.890,1,0.0280,113.889,0.7850
143678,0,NaN,0.732000,0.428,0.276,3oEHQmhvFLiE7ZYES0ulzv,0.000005,4,0.1170,-11.729,1,0.0312,99.195,0.0382


In [5]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df[['acousticness','danceability','energy','instrumentalness','key',\
                                           'liveness','loudness','mode','speechiness','tempo','valence']])
df_normalized = pd.DataFrame(np_scaled)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
df_normalized.columns = ['acousticness','danceability','energy','instrumentalness','key',\
                                           'liveness','loudness','mode','speechiness','tempo','valence']

In [7]:
df_normalized['id'] = df['id']
df_normalized.tail()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,id
143674,0.000158,0.640404,0.958,0.702000,0.818182,0.090070,0.858171,0.0,0.035088,0.560230,0.8140,4WYZ2dp1BhnPovdiXsvU2S
143675,0.082329,0.512121,0.849,0.000000,0.454545,0.799398,0.799972,1.0,0.107327,0.531974,0.1540,1biP5S4CFMNfLA0rceCmRF
143676,0.000824,0.687879,0.942,0.000030,0.818182,0.065797,0.874327,1.0,0.049845,0.528021,0.8850,3DZpiHNi3wQuoIMuiJ5jxJ
143677,0.034036,0.842424,0.869,0.000522,0.818182,0.146439,0.852045,1.0,0.028896,0.455695,0.7850,3MYe1siYBRujEK5fy9aa3h
143678,0.734940,0.432323,0.276,0.000005,0.363636,0.117352,0.760101,1.0,0.032198,0.396901,0.0382,3oEHQmhvFLiE7ZYES0ulzv


In [8]:
df_normalized,last_row=df_normalized.drop(df_normalized.tail(1).index),df_normalized.tail(1)
last_row

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,id
143678,0.73494,0.432323,0.276,0.000005,0.363636,0.117352,0.760101,1.0,0.032198,0.396901,0.0382,3oEHQmhvFLiE7ZYES0ulzv


In [9]:
last_row['acousticness']
df_normalized['score'] = ( 
    df_normalized['acousticness'].apply(lambda row: np.abs(row - last_row['acousticness'])) + \
    df_normalized['danceability'].apply(lambda row: np.abs(row - last_row['danceability'])) + \
    df_normalized['energy'].apply(lambda row: np.abs(row - last_row['energy'])) + \
    df_normalized['instrumentalness'].apply(lambda row: np.abs(row - last_row['instrumentalness'])) + \
    df_normalized['key'].apply(lambda row: np.abs(row - last_row['key'])) + \
    df_normalized['liveness'].apply(lambda row: np.abs(row - last_row['liveness'])) + \
    df_normalized['loudness'].apply(lambda row: np.abs(row - last_row['loudness'])) + \
    df_normalized['mode'].apply(lambda row: np.abs(row - last_row['mode'])) + \
    df_normalized['speechiness'].apply(lambda row: np.abs(row - last_row['speechiness'])) + \
    df_normalized['tempo'].apply(lambda row: np.abs(row - last_row['tempo'])) + \
    df_normalized['valence'].apply(lambda row: np.abs(row - last_row['valence']))                                    
)

In [10]:
df_normalized.tail()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,id,score
143673,0.156627,0.780808,0.694,0.009980,0.454545,0.094082,0.839464,0.0,0.032508,0.380192,0.829,3fBzXNIRUnHl80OFhB0jo7,3.356133
143674,0.000158,0.640404,0.958,0.702000,0.818182,0.090070,0.858171,0.0,0.035088,0.560230,0.814,4WYZ2dp1BhnPovdiXsvU2S,4.848774
143675,0.082329,0.512121,0.849,0.000000,0.454545,0.799398,0.799972,1.0,0.107327,0.531974,0.154,1biP5S4CFMNfLA0rceCmRF,2.444241
143676,0.000824,0.687879,0.942,0.000030,0.818182,0.065797,0.874327,1.0,0.049845,0.528021,0.885,3DZpiHNi3wQuoIMuiJ5jxJ,3.271589
143677,0.034036,0.842424,0.869,0.000522,0.818182,0.146439,0.852045,1.0,0.028896,0.455695,0.785,3MYe1siYBRujEK5fy9aa3h,3.088994


In [23]:
df_normalized['class'] = df_normalized['score'].apply(lambda row: True if row < 1.5 else False)
df_normalized.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,id,score,class
0,0.573293,0.647475,0.665,0.001070,0.636364,0.159478,0.821355,1.0,0.027141,0.552236,0.677,1vPk6ZTflnvm8LgtXzPDrT,1.942162,False
1,0.720884,0.679798,0.405,0.000268,0.636364,0.122367,0.797988,1.0,0.032301,0.624026,0.652,6OpIhef0IbK0tkEU2m9c4T,1.547450,False
2,0.247992,0.732323,0.488,0.000000,0.636364,0.080341,0.792114,1.0,0.034675,0.496523,0.456,36qxb23eWvnQiJWEMgWYO6,1.860603,False
3,0.516064,0.691919,0.503,0.000028,1.000000,0.082447,0.831827,1.0,0.042931,0.375806,0.807,1FIlg68WTgn0xGL8vPy5Rj,2.249115,False
4,0.790161,0.614141,0.331,0.000000,0.636364,0.324975,0.776431,1.0,0.029928,0.552692,0.709,7oBAMX1mPa2qiqmSunLirZ,1.617585,False


In [22]:
df_normalized['class'].value_counts()

False    129924
True      13754
Name: class, dtype: int64

## Data Pre-Processing

In [13]:
data = df_normalized.drop(["class", "score","id"], axis=1)
#feature_names = data.columns
data.head()
target = df_normalized["class"]
print(data.shape, target.shape)

(143678, 11) (143678,)


In [14]:


X_train, X_test, y_train, y_test = train_test_split(
    data, target, random_state=20, stratify=target)
# X_scaler = StandardScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [15]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='selu', input_dim=11))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1200      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 102       
Total params: 19,002
Trainable params: 19,002
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
#     X_train_scaled,  Test 2 - Removed scaling.  This step was performed on data BEFORE being entered into machine
    X_train,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=1
)

Epoch 1/20
107758/107758 [==============================] - 5s 49us/step - loss: 0.0028 - acc: 0.9997
Epoch 2/20
107758/107758 [==============================] - 5s 46us/step - loss: 0.0015 - acc: 0.9999
Epoch 3/20
107758/107758 [==============================] - 5s 45us/step - loss: 0.0015 - acc: 0.9999
Epoch 4/20
107758/107758 [==============================] - 5s 50us/step - loss: 0.0015 - acc: 0.9999
Epoch 5/20
107758/107758 [==============================] - 5s 47us/step - loss: 0.0015 - acc: 0.9999
Epoch 6/20
107758/107758 [==============================] - 5s 47us/step - loss: 0.0015 - acc: 0.9999
Epoch 7/20
107758/107758 [==============================] - 5s 49us/step - loss: 0.0015 - acc: 0.9999
Epoch 8/20
107758/107758 [==============================] - 5s 48us/step - loss: 0.0015 - acc: 0.9999
Epoch 9/20
107758/107758 [==============================] - 5s 49us/step - loss: 0.0015 - acc: 0.9999
Epoch 10/20
107758/107758 [==============================] - 5s 46us/step - loss: 

## Quantify our Trained Model

In [17]:
#For second test, sub'd X_test_scaled for _test
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_categorical, verbose=2) 
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.0017950071258375003, Accuracy: 0.9998886414253898


## Make Predictions

In [18]:
#For second test, sub'd X_test_scaled for _test
encoded_predictions = model.predict_classes(X_test[:100])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [19]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:100])}")

Predicted classes: [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Actual Labels: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [20]:
name = input("Provide Name for Saved File: ")
# Save the model
folder = "Saved Models"
model.save(f'{folder}\{name}.h5')

Provide Name for Saved File: DeepLearning_Test3_acc99


KeyboardInterrupt: 